In [1]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman parameters
q = 7919  # Small prime for demo; use large prime in real systems
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        print(data.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.partial_fit(self.X_train, self.y_train,classes=[0,1])
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load and preprocess dataset
    X, y = make_classification(n_samples=1000, n_features=30, n_classes=2,
                           n_informative=15, n_redundant=5, flip_y=0.1,
                           random_state=42)


    print(X.shape)
    # Normalize features
    

    # Split data across nodes
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    # Initialize nodes
    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        shared_key = sender.compute_shared_key(receiver.public_key)
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        current_holder = (current_holder + 1) % num_nodes

        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)

(1000, 30)
(334, 30)
(333, 30)
(333, 30)

🎯 Initial Training at Node 0
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 0.7164179104477612
Node 1 Test Accuracy: 0.7164179104477612
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 0.7014925373134329
Node 1 Test Accuracy: 0.7611940298507462
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted 